# Building a Spam Filter Using Naive Bayes Algorithm

In this project, we're going to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. Our goal is to write a program that classifies new messages with an accuracy greater than 80% — so we expect that more than 80% of the new messages will be classified correctly as spam or ham (non-spam).

To train the algorithm, we'll use a dataset of 5,572 SMS messages that are already classified by humans. The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). The data collection process is described in more details on this page, where you can , find some of the papers authored by Tiago A. Almeida and José María Gómez Hidalgo.

### Exploring the dataset

In [1]:
import pandas as pd
import numpy as np

In [2]:
messages=pd.read_csv("SMSSpamCollection",sep="\t",header=None)

In [3]:
messages.columns=['label','text']

In [4]:
messages.columns

Index(['label', 'text'], dtype='object')

In [5]:
messages

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [6]:
# finding % of spams and hams
messages['label'].value_counts(normalize=True)*100

ham     86.593683
spam    13.406317
Name: label, dtype: float64

### Training & Test Set

Once our spam filter is done, we'll need to test how good it is with classifying new messages. To test the spam filter, we're first going to split our dataset into two categories: Training and Test sets. We're going to keep 80% of our dataset for training, and 20% for testing

In [7]:
# randomizing the dataset and splitting into training and test
messages=messages.sample(frac=1,random_state=1)

In [8]:
# Calculate index for split
training_test_index = round(len(messages) * 0.8)

# Training/Test split
training = messages[:training_test_index].reset_index(drop=True)


In [9]:
test= messages[training_test_index:].reset_index(drop=True)

In [10]:
training

,label,text
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...
...,...,...
4453,ham,"Sorry, I'll call later in meeting any thing re..."
4454,ham,Babe! I fucking love you too !! You know? Fuck...
4455,spam,U've been selected to stay in 1 of 250 top Bri...
4456,ham,Hello my boytoy ... Geeee I miss you already a...


In [11]:
test

,label,text
0,ham,Later i guess. I needa do mcat study too.
1,ham,But i haf enuff space got like 4 mb...
2,spam,Had your mobile 10 mths? Update to latest Oran...
3,ham,All sounds good. Fingers . Makes it difficult ...
4,ham,"All done, all handed in. Don't know if mega sh..."
...,...,...
1109,ham,"We're all getting worried over here, derek and..."
1110,ham,Oh oh... Den muz change plan liao... Go back h...
1111,ham,CERI U REBEL! SWEET DREAMZ ME LITTLE BUDDY!! C...
1112,spam,Text & meet someone sexy today. U can find a d...


In [12]:
print(training['label'].value_counts(normalize=True)*100,"\n")
print(test['label'].value_counts(normalize=True)*100)

ham     86.54105
spam    13.45895
Name: label, dtype: float64 

ham     86.804309
spam    13.195691
Name: label, dtype: float64


We can see that the distribution of spam and ham messages has been retained in the both the training and test datasets

### Data Cleaning

To calculate all the probabailities required by the algorithm, we'll first need to perform a bit of data cleaning to bring the data in a format that will allow us to extract easily all the information we need. We will first convert all letters to lower case and then remove all punctuation. 

Essentially, we want to bring data to this format: ![img](https://dq-content.s3.amazonaws.com/433/cpgp_dataset_3.png)

In [13]:
training['text']=training['text'].str.lower().str.replace("\W",' ')
training['text']

0                            yep  by the pretty sculpture
1           yes  princess  are you going to make me moan 
2                              welp apparently he retired
3                                                 havent 
4       i forgot 2 ask ü all smth   there s a card on ...
                              ...                        
4453    sorry  i ll call later in meeting any thing re...
4454    babe  i fucking love you too    you know  fuck...
4455    u ve been selected to stay in 1 of 250 top bri...
4456    hello my boytoy     geeee i miss you already a...
4457                             wherre s my boytoy      
Name: text, Length: 4458, dtype: object

### Creating the vocabulary

Let's now move to creating the vocabulary, which in this context means a list with all the unique words in our training set.

In [14]:
#creating a list with all the words
vocabulary=[]
for i in training['text'].str.split():
    for j in i:
        vocabulary.append(j)
    

In [15]:
set_vocab=set(vocabulary)

In [16]:
len(set_vocab)

7783

In [17]:
len(vocabulary)

72427

In [18]:
vocabulary=list(set_vocab)

type(vocabulary)


It looks like there are 7,783 unique words in all the messages of our training set.

In [19]:
len(vocabulary)

7783

### The Final Training Set

We're now going to use the vocabulary we just created to make the data transformation we want.


In [20]:
training['text']=training['text'].str.split()

In [21]:
training

,label,text
0,ham,"[yep, by, the, pretty, sculpture]"
1,ham,"[yes, princess, are, you, going, to, make, me,..."
2,ham,"[welp, apparently, he, retired]"
3,ham,[havent]
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,..."
...,...,...
4453,ham,"[sorry, i, ll, call, later, in, meeting, any, ..."
4454,ham,"[babe, i, fucking, love, you, too, you, know, ..."
4455,spam,"[u, ve, been, selected, to, stay, in, 1, of, 2..."
4456,ham,"[hello, my, boytoy, geeee, i, miss, you, alrea..."


In [22]:
dict_vocab={}
for i in vocabulary:
    count_list=[]
    for j in training['text']:
        count_list.append(j.count(i))
    dict_vocab[i]=count_list
    
        

In [23]:
training_final=pd.DataFrame(dict_vocab)

In [24]:
training_final

,listening,worry,comment,mark,630,gain,lists,download,tt,spl,...,nagar,necklace,ukp,nobbing,removal,appreciate,vouch4me,stamps,celebration,kerala
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4453,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4454,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4455,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4456,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
training_final['label']=training['label']

In [26]:
training_final

,listening,worry,comment,mark,630,gain,lists,download,tt,spl,...,necklace,ukp,nobbing,removal,appreciate,vouch4me,stamps,celebration,kerala,label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ham
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ham
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ham
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ham
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ham
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4453,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ham
4454,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ham
4455,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,spam
4456,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ham


In [27]:
training_final.insert(0, 'label_text', training['label'], allow_duplicates = False)

In [28]:
training_final.drop('label',axis=1,inplace=True)

In [29]:
training_final.insert(1,'SMS',training['text'],allow_duplicates=False)

### Calculating Constants of the Naive Bayes Algorithm

We're now done with cleaning the training set, and we can begin creating the spam filter. The Naive Bayes algorithm will need to answer these two probability questions to be able to classify new messages

\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam)
\end{equation}

\begin{equation}
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}


Also, to calculate P(wi|Spam) and P(wi|Ham) inside the formulas above, we'll need to use these equations:

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
\end{equation}

\begin{equation}
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

Some of the terms in the four equations above will have the same value for every new message. We can calculate the value of these terms once and avoid doing the computations again when a new messages comes in. Below, we'll use our training set to calculate:

- P(Spam) and P(Ham)
- NSpam, NHam, NVocabulary

We'll also use Laplace smoothing and set $\alpha = 1$.

In [30]:
### no of total words in ham messages
nwords_ham=(training_final[training_final['label_text']=='ham'].sum())[2:].sum()

In [31]:
### no of total words in spam messages
nwords_spam=(training_final[training_final['label_text']=='spam'].sum())[2:].sum()

In [32]:
### probability of spam messages
p_spam=len(training_final[training_final['label_text']=='spam'])/len(training_final)

### probability of ham messages
p_ham=len(training_final[training_final['label_text']=='ham'])/len(training_final)

In [33]:
### Assigning laplace smoothning paramter alpha
alpha = 1

In [34]:
# # Isolating spam and ham messages first
spam_messages = training_final[training_final['label_text'] == 'spam']
ham_messages = training_final[training_final['label_text'] == 'ham']



### Calculating Parameters of the Naive Bayes Algorithm

Now that we have the constant terms calculated above, we can move on with calculating the parameters $P(w_i|Spam)$ and $P(w_i|Ham)$. Each parameter will thus be a conditional probability value associated with each word in the vocabulary.

The parameters are calculated using the formulas: ![](math.svg)
![](math_ham.svg)

In [35]:
ham_messages

,label_text,SMS,listening,worry,comment,mark,630,gain,lists,download,...,nagar,necklace,ukp,nobbing,removal,appreciate,vouch4me,stamps,celebration,kerala
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4452,ham,"[how, about, clothes, jewelry, and, trips]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4453,ham,"[sorry, i, ll, call, later, in, meeting, any, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4454,ham,"[babe, i, fucking, love, you, too, you, know, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4456,ham,"[hello, my, boytoy, geeee, i, miss, you, alrea...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
n_vocab=len(vocabulary)


In [37]:
ham_word_list=ham_messages.iloc[:,2:].sum()
ham_messages_unique=(ham_word_list+alpha)/(nwords_ham+(alpha*n_vocab))

In [38]:
ham_messages_unique

listening      0.000077
worry          0.000292
comment        0.000031
mark           0.000108
630            0.000046
                 ...   
appreciate     0.000123
vouch4me       0.000015
stamps         0.000046
celebration    0.000031
kerala         0.000062
Length: 7783, dtype: float64

In [39]:
ham_dict=dict(ham_messages_unique)

In [40]:
spam_messages

,label_text,SMS,listening,worry,comment,mark,630,gain,lists,download,...,nagar,necklace,ukp,nobbing,removal,appreciate,vouch4me,stamps,celebration,kerala
16,spam,"[freemsg, why, haven, t, you, replied, to, my,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18,spam,"[congrats, 2, mobile, 3g, videophones, r, your...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
56,spam,"[free, message, activate, your, 500, free, tex...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60,spam,"[call, from, 08702490080, tells, u, 2, call, 0...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
61,spam,"[someone, has, conacted, our, dating, service,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4437,spam,"[congratulations, you, ve, won, you, re, a, wi...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4439,spam,"[win, the, newest, harry, potter, and, the, or...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4443,spam,"[someone, u, know, has, asked, our, dating, se...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4449,spam,"[your, chance, to, be, on, a, reality, fantasy...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
spam_word_list=spam_messages.iloc[:,2:].sum()
spam_messages_unique=(spam_word_list+alpha)/(nwords_spam+(alpha*n_vocab))

In [42]:
spam_messages_unique

listening      0.000087
worry          0.000044
comment        0.000044
mark           0.000044
630            0.000044
                 ...   
appreciate     0.000044
vouch4me       0.000087
stamps         0.000044
celebration    0.000044
kerala         0.000044
Length: 7783, dtype: float64

In [43]:
spam_dict=dict(spam_messages_unique)

### Classifying a New Message

Now that we have all our parameters calculated, we can start creating the spam filter. The spam filter can be understood as a function that:

- Takes in as input a new message (w1, w2, ..., wn).
- Calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn).
- Compares the values of P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn), and:
 - If P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn), then the message is classified as ham.
 - If P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn), then the message is classified as spam.
 - If P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn), then the algorithm may request human help.

In [44]:
test

,label,text
0,ham,Later i guess. I needa do mcat study too.
1,ham,But i haf enuff space got like 4 mb...
2,spam,Had your mobile 10 mths? Update to latest Oran...
3,ham,All sounds good. Fingers . Makes it difficult ...
4,ham,"All done, all handed in. Don't know if mega sh..."
...,...,...
1109,ham,"We're all getting worried over here, derek and..."
1110,ham,Oh oh... Den muz change plan liao... Go back h...
1111,ham,CERI U REBEL! SWEET DREAMZ ME LITTLE BUDDY!! C...
1112,spam,Text & meet someone sexy today. U can find a d...


In [45]:
import re
def classify(text):
    text=re.sub('\W',' ',text)
    text=text.lower()
    text=text.split()
    p_w_spam=p_spam
    p_w_ham=p_ham
    for i in text:
        if i in vocabulary:
            p_w_spam*=spam_dict[i]
            p_w_ham*=ham_dict[i]
    if p_w_spam>p_w_ham:
        flag="spam"
    elif p_w_ham>p_w_spam:
        flag="ham"
    else:
        flag='human'
    return flag

test['spam_or_ham']=test['text'].apply(classify)

        

### Measuring the Spam Filter's Accuracy

The two results above look promising, but let's see how well the filter does on our test set, which has 1,114 messages.

In [46]:
test['diff']=np.where(test['spam_or_ham']==test['label'],"correct","incorrect")

In [47]:
results=test['diff'].value_counts()

In [48]:
results

correct      1100
incorrect      14
Name: diff, dtype: int64

In [49]:
accuracy=results['correct']/(results['correct']+results['incorrect'])
print("Accuracy =",accuracy*100)

Accuracy = 98.74326750448833


The accuracy is close to 98.74%, which is really good. Our spam filter looked at 1,114 messages that it hasn't seen in training, and classified 1,100 correctly.

### Conclusion

In this project, we managed to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. The filter had an accuracy of 98.74% on the test set we used, which is a pretty good result. Our initial goal was an accuracy of over 80%, and we managed to do way better than that.